# LOTUS demo

## APIs

In [4]:
import bigframes.pandas as bpd
from bigframes.ml.llm import GeminiTextGenerator

data = {
    "Course Name": [
        "Probability and Random Processes",
        "Optimization Methods in Engineering",
        "Digital Design and Integrated Circuits",
        "Computer Security",
        "Operating Systems and Systems Programming",
        "Compilers",
        "Computer Networks",
        "Deep Learning",
        "Graphics",
        "Databases",
        "Art History",
    ]
}
df = bpd.DataFrame(data)
 
model = GeminiTextGenerator(model_name="gemini-1.5-flash-001")

### 1. `sem_filter`

In [5]:
predict_df = df.sem_filter("{Course Name} requires a lot of math", model)
predict_df

/usr/local/google/home/chelsealin/src/bigframes/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,True 0.95,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
1,True 0.95,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
2,True 0.95,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
3,False 0.85,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
4,False 0.90,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
5,True 0.85,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
6,False 0.85,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
7,True 0.95,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
8,True 0.80,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
9,False 0.95,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...


## Optimizations

### Cascade Models

In [2]:
# TODO